In [1]:
!pip install rank_bm25

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!tar -xzf /content/drive/MyDrive/Isabelle2024_linux.tar.gz -C /content/
print('Extraction finished.')

Extraction finished.


In [4]:
import os
os.environ['ISABELLE_DIRPATH'] = os.path.abspath('/content/Isabelle2024')

import nest_asyncio
nest_asyncio.apply()

In [5]:
!git clone https://github.com/lanzhang128/multi_agent_autoformalization.git
%cd multi_agent_autoformalization
!mkdir test_results

fatal: destination path 'multi_agent_autoformalization' already exists and is not an empty directory.
/content/multi_agent_autoformalization
mkdir: cannot create directory ‘test_results’: File exists


Definition of Softmax Function

In [6]:
informal = (
    "Definition of Softmax Function: Formally, the standard (unit) softmax function "
    "\\(\\sigma\\colon \\mathbb{R}^K \\to (0, 1)^K\\), where \\(K \\ge 1\\), takes a vector "
    "\\(\\mathbf{z} = (z_1, \\dotsc, z_K) \\in \\mathbb{R}^K\\) and computes each component of vector "
    "\\(\\sigma(\\mathbf{z}) \\in (0, 1)^K\\) with \\(\\sigma(\\mathbf{z})_i = \\frac{e^{z_i}}{\\sum_{j=1}^K e^{z_j}}\\,.\\)"
)

Set target language to Isabelle/HOL.

In [7]:
formal_language = 'Isabelle/HOL'

First instantiate an OpenAI model using GPT-4o as the backend LLM for agents.

In [8]:
from llm import OpenAILLM

gpt = OpenAILLM(
    api_key='...',
    model='gpt-4o')

Instantiate an autoformalization agent and perform zero-shot autoformalization.

In [9]:
from agent import AutoformalizationAgent

agent_auto = AutoformalizationAgent(
    llm=gpt,
    formal_language=formal_language)
zero_formalization, _ = agent_auto(
    informal_statement=informal)
print(zero_formalization)

definition softmax :: "real list ⇒ real list" where
  "softmax z = 
    let exp_z = map exp z;
        sum_exp_z = sum_list exp_z
    in map (λzi. exp zi / sum_exp_z) z"



Instantiate a hard critique agent to check the syntactic correctness of the previous formalization.

In [10]:
from agent import HardCritiqueAgent
agent_hard = HardCritiqueAgent(
    formal_language=formal_language,
    file_dir="./test_results")
correctness, error_details = agent_hard(
    formalization=zero_formalization,
    file_prefix="zero")
print(correctness)
print(error_details)

logging file is at: ./test_results/isabelle.log
Isabelle server started with info: server "test" = 127.0.0.1:40879 (password "8e4c3b73-63cc-4a6f-9f63-6b90486f91a3")
 in 2.31s.
Isabelle server session 2d87384e-615f-4e0f-9d9b-1ebe4d38c0e9 started in 13.19s.
Isabelle server use_theory ended in 2.54s.
False
Identified error on line: 4
Error message: Undefined type name: "real"\<^here>
Failed to parse type




Instantiate a tool agent for import retrieval.

In [11]:
from agent import ImportRetrievalAgent
agent_imports = ImportRetrievalAgent(
    formal_language=formal_language,
    retriever="bm25")
import_formalization = agent_imports(
    formalization=zero_formalization, top_n=1)
print(import_formalization)

imports
Main
"HOL.Complex"
begin
definition softmax :: "real list ⇒ real list" where
  "softmax z = 
    let exp_z = map exp z;
        sum_exp_z = sum_list exp_z
    in map (λzi. exp zi / sum_exp_z) z"

end


Test the formalization again.

In [12]:
correctness, error_details = agent_hard(
    formalization=import_formalization,
    file_prefix="imp")
print(correctness)
print(error_details)

Isabelle server use_theory ended in 2.27s.
False
Identified error on line: 6
Error message: Inner syntax error\<^here>
Failed to parse prop




Instantiate a formal refinement agent to try to fix the errors.

In [13]:
from agent import FormalRefinementAgent
agent_formal = FormalRefinementAgent(
    llm=gpt,
    formal_language=formal_language,
    category="none")
detailed_refinement, response = agent_formal(
    informal_statement=informal,
    refinement_mode="detailed",
    formalization=import_formalization,
    correctness=correctness,
    error_details=error_details)
print(detailed_refinement)

theory Softmax
  imports Main
begin

definition softmax :: "real list ⇒ real list" where
  "softmax z = 
    let exp_z = map exp z;
        sum_exp_z = listsum exp_z
    in map (λzi. exp zi / sum_exp_z) z"

end



Alternativel instantiate a soft critique agent to focus on one aspect.

In [14]:
aspect_description='whether the formalized code involves all mathematical concepts in the natural language statement.'

In [15]:
from agent import SoftCritiqueAgent

agent_soft = SoftCritiqueAgent(
    llm=gpt,
    formal_language=formal_language,
    aspect_description=aspect_description)
aspect_evaluation, _ = agent_soft(
    informal_statement=informal,
    formalization=import_formalization)
print(aspect_evaluation)

Explanation: The natural language statement describes the softmax function, which maps a vector from \(\mathbb{R}^K\) to \((0, 1)^K\) using the formula \(\sigma(\mathbf{z})_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}\). The formal code defines a function `softmax` that takes a list of real numbers and returns a list of real numbers. It uses the `map` function to apply the exponential function to each element of the list, calculates the sum of these exponentials, and then maps each element to its softmax value using the given formula. The code correctly captures the essential components of the softmax function: it computes the exponential of each element, sums these exponentials, and divides each exponential by the sum to produce the output list. The code also correctly handles the input and output types, mapping from a list of real numbers to another list of real numbers, which corresponds to the vector transformation described in the natural language statement. Therefore, the formalized 

Instantiate an informal refinement agent to refine the formal code based on the evaluation of the relevant aspect.

In [16]:
from agent import InformalRefinementAgent

agent_informal = InformalRefinementAgent(
    llm=gpt,
    formal_language=formal_language)
informal_refinement, _ = agent_informal(
    informal_statement=informal,
    formalization=import_formalization,
    aspect_description=aspect_description,
    aspect_evaluation=aspect_evaluation)
print(informal_refinement)

theory Softmax
  imports
    Main
    "HOL.Complex"
begin

definition softmax :: "real list ⇒ real list" where
  "softmax z = 
    let exp_z = map exp z;
        sum_exp_z = sum_list exp_z
    in map (λzi. exp zi / sum_exp_z) z"

end



In [17]:
agent_hard.theorem_prover.terminate()

Isabelle server 127.0.0.1:40879 shutdown.
